In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris, fetch_california_housing

import matplotlib.pyplot as plt
import seaborn as sns

In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris, fetch_california_housing
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Rerading the file

In [3]:
df= pd.read_csv('../Data/Raw/health_lifestyle_classification.csv')

# Dropping columns that we consider unnecessary and then dropping NA

In [4]:
df.columns

Index(['survey_code', 'age', 'gender', 'height', 'weight', 'bmi',
       'bmi_estimated', 'bmi_scaled', 'bmi_corrected', 'waist_size',
       'blood_pressure', 'heart_rate', 'cholesterol', 'glucose', 'insulin',
       'sleep_hours', 'sleep_quality', 'work_hours', 'physical_activity',
       'daily_steps', 'calorie_intake', 'sugar_intake', 'alcohol_consumption',
       'smoking_level', 'water_intake', 'screen_time', 'stress_level',
       'mental_health_score', 'mental_health_support', 'education_level',
       'job_type', 'occupation', 'income', 'diet_type', 'exercise_type',
       'device_usage', 'healthcare_access', 'insurance', 'sunlight_exposure',
       'meals_per_day', 'caffeine_intake', 'family_history', 'pet_owner',
       'electrolyte_level', 'gene_marker_flag', 'environmental_risk_score',
       'daily_supplement_dosage', 'target'],
      dtype='object')

In [5]:
columns_to_drop=['gene_marker_flag', 'income','survey_code', 'bmi_estimated', 'bmi_scaled','bmi_corrected','insurance','occupation','electrolyte_level','education_level']

In [6]:
df_2=df.drop(columns_to_drop,axis=1)
df_2=df_2.dropna()
# df_2.info()

# Identifying object columns and dividing them into ordinals and not ordinals

In [7]:
df_2.select_dtypes('object').columns

Index(['gender', 'sleep_quality', 'alcohol_consumption', 'smoking_level',
       'mental_health_support', 'job_type', 'diet_type', 'exercise_type',
       'device_usage', 'healthcare_access', 'sunlight_exposure',
       'caffeine_intake', 'family_history', 'pet_owner', 'target'],
      dtype='object')

In [8]:
df_2['device_usage'].unique()

array(['Moderate', 'High', 'Low'], dtype=object)

In [9]:
object_col_non_ord=['gender', 'alcohol_consumption', 'mental_health_support', 'job_type', 'diet_type', 'exercise_type','caffeine_intake', 'family_history', 'pet_owner']
object_col_ord=['sleep_quality','smoking_level','device_usage','healthcare_access','sunlight_exposure']

In [10]:
device_usage_map = {'Low': 0,
    'Moderate': 1,
    'High': 2
}  
df_2['device_usage'] = df_2['device_usage'].map(device_usage_map)

In [11]:
quality_mapping = {
    'Poor': 0,
    'Fair': 1,
    'Good': 2,
    'Excellent': 3
}
df_2['sleep_quality'] = df_2['sleep_quality'].map(quality_mapping)

In [12]:
smoking_map = {
    'Non-smoker': 0,
    'Light': 1,
    'Heavy': 2
}
df_2['smoking_level'] = df_2['smoking_level'].map(smoking_map)

In [13]:
healthcare_access_map = {
    'Poor': 0,
    'Moderate': 1,
    'Good': 2
}
df_2['healthcare_access'] = df_2['healthcare_access'].map(healthcare_access_map)

In [14]:
sunlight_exposure_map = {
    'Low': 0,
    'Moderate': 1,
    'High': 2
}
df_2['sunlight_exposure'] = df_2['sunlight_exposure'].map(sunlight_exposure_map)

# Training the encoder

In [16]:
ohe = OneHotEncoder(sparse_output=False)
ohe.fit(df_2[object_col_non_ord])

,categories,'auto'
,drop,None
,sparse_output,False
,dtype,<class 'numpy.float64'>
,handle_unknown,'error'
,min_frequency,None
,max_categories,None
,feature_name_combiner,'concat'


# Doing the Split

In [17]:
features = df_2.drop(columns='target')
target = df_2["target"]
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.20, random_state=0)

In [18]:
X_train_num=X_train.drop(object_col_non_ord, axis=1)
X_test_num= X_test.drop(object_col_non_ord, axis=1)

# We are creating the new DF after enconding and we are overwriting the X_train and X_test to avoid doing the split again

In [ ]:
X_train_trans_np = ohe.transform(X_train[object_col_non_ord])
X_train_trans_df = pd.DataFrame(X_train_trans_np, columns=ohe.get_feature_names_out(), index=X_train.index)
X_train=pd.concat([X_train_trans_df,X_train_num],axis=1)
# X_train.info()

In [ ]:
X_test_trans_np = ohe.transform(X_test[object_col_non_ord])
X_test_trans_df = pd.DataFrame(X_test_trans_np, columns=ohe.get_feature_names_out(), index=X_test.index)
X_test=pd.concat([X_test_trans_df,X_test_num],axis=1)
# X_test.info()

# Applying the minmaxscaler

In [43]:
normalizer = MinMaxScaler()
normalizer.fit(X_train)

,feature_range,"(0, ...)"
,copy,True
,clip,False


In [44]:
X_train_norm = normalizer.transform(X_train)
X_test_norm = normalizer.transform(X_test)

X_train = pd.DataFrame(X_train_norm, columns=X_train.columns, index=X_train.index )
X_test = pd.DataFrame(X_test_norm, columns=X_test.columns, index=X_test.index)

# Testing the model

In [45]:
knn = KNeighborsClassifier(n_neighbors=9)
knn.fit(X_train, y_train)
print(f"The accuracy of the model is {knn.score(X_test, y_test)*100: .2f}%")

The accuracy of the model is  66.41%


In [ ]:
corr=np.abs(df_2.corr()) # corr(x,y) = corr(y, x), corr(x,x) = 1
#Set up mask for triangle representation
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(10, 10))
# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask,  vmax=1,square=True, linewidths=.5, cbar_kws={"shrink": .5},annot = corr)
plt.show()

In [ ]:
# df_2.to_csv('../Data/Clean/Clean_data.csv',index=False)